In [5]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

import time
import os
import numpy as np

import tensorflow as tf

modelfileName = "CS286-Word2Vec.model"
training_size_arr_actual = [0, 0, 0]
training_size_arr_simulated = [0, 0, 0, 0]

dirNameInput = 'Word2VecData_Input'
dirNameOutput = 'Word2VecData_Output'


Word2Vec_model = None

X = None
Y = None
x_train = None
y_train = None
x_test = None
y_test = None

model = None


In [6]:
def createDataFiles():
    import time
    start = time.time()
    stride = 2
    messageSize = 5
    
    # For Simulated data
    fileInput = open(dirNameInput + "/Word2VecDataSimulated.txt", "w")
    fileOutput = open(dirNameOutput + "/Word2VecDataSimulated.txt", "w")
    fileNamesSimulated = ['Speed20_40.txt', 'Speed40_60.txt']
    arrSimulated = [1, 2]

    for i in range(len(fileNamesSimulated)):
        with open(fileNamesSimulated[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]

        cntCntr = 0
        while cntCntr < len(content):
            iteration = 0
            currMessagePackets = ""

            while iteration < messageSize and cntCntr < len(content):
                # Remove the noise from the files
                DATA = content[cntCntr][15:38]

                # this is the whole message
                parts = DATA.split()

                tempo = ""
                for j in range(8):
                    if j >= len(parts) or parts[j] == "  ":
                        tempo += "00"
                    else:
                        tempo += parts[j]

                currMessagePackets += str(tempo) + " "
                cntCntr += 1
                iteration += 1

            if cntCntr < len(content):
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(arrSimulated[i]) + "\n")

    fileInput.close()
    fileOutput.close()
    # To only work with actual data end commenting till here
    end = time.time()

    print("Awesome !!! File processing done !!!")
    print("Total Time for file processing ---> ", end - start)

In [7]:
class IteratingClass:
    def __init__(self, dirName):
        self.dirName = dirName

    def __iter__(self):
        for fName in os.listdir(self.dirName):
            for line in open(os.path.join(self.dirName, fName)):
                yield line.split()

def createWord2VecModels():
    start = time.time()
    dataDirec = IteratingClass(dirNameInput)
    model = Word2Vec(dataDirec, size=200, window=5, min_count=1, workers=8)
    model.save(modelfileName)
    end = time.time()
    print("\n\nTraining Successful for Word2Vec Model!!!")
    print("Total Time for Word2Vec model -> ", (end - start))


In [8]:
def train():
    createDataFiles()
    createWord2VecModels()



In [10]:
train()

Awesome !!! File processing done !!!
Total Time for file processing --->  0.04319357872009277


Training Successful for Word2Vec Model!!!
Total Time for Word2Vec model ->  0.0533289909362793


In [19]:

# Convert the data into Numpy Arrays: It will take a lot of time
start = time.time()
Word2Vec_model = Word2Vec.load(modelfileName)
#x_train = None
def getNumpyArray():
    global X
    global Y
    global dirNameInput
    X = None
    Y = None
    print("Processing started ")
    result = []
    for fName in os.listdir(dirNameInput):
        for line in open(os.path.join(dirNameInput, fName)):
            parts = line.split()
            for i in range(0, len(parts)):
                result.append(np.array(Word2Vec_model.wv[parts[i]]))
    X = np.array(result)
                
                
    print("Processing started for output ")
    resultY = []
    for fName in os.listdir(dirNameOutput):
        for line in open(os.path.join(dirNameOutput, fName)):
            currV = int(line)
            for i in range(0,5):
                resultY.append(np.array(currV))
    
    Y = np.array(resultY)
    print(X.shape)    
    print(Y.shape)    
    end = time.time()
    print("total time -> ", end -start)





#    LSTM

In [23]:
getNumpyArray()



X = X.reshape(5985,200,1)
Y = Y.reshape(5985,1, 1)





Processing started 
Processing started for output 
(5985, 200)
(5985,)
total time ->  68.5667884349823


In [24]:
def executeModel(eph):
    start = time.time()
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM((1), input_shape=(None, 5, 200), return_sequences=True))
    #model.add(tf.keras.layers.LSTM(5 ,input_shape=(200,1)))
    #model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


    print ('Compiling...')
    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size = 100 , epochs=eph, verbose = 1)

    val_loss, val_acc = model.evaluate(X_test, y_test)
    print(val_loss)
    print(val_acc)

    end = time.time()
    print("Total Time -> ", (end - start))
    
    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(val_loss)
    print(val_acc)



In [27]:
#X = X.reshape(26777,200,1)
X = X[:1000]
Y = Y[:1000]
print(X.shape)
print(Y.shape)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
executeModel(10)

(1000, 200, 1)
(1000, 1, 1)


ValueError: Shapes must be equal rank, but are 3 and 2

In [26]:
start = time.time()
#executeModel(50)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Embedding(input_dim = 188, output_dim = 50, input_length = X.shape[1]))
model.add(tf.keras.layers.LSTM(200,activation='sigmoid', input_shape = (200,1), recurrent_activation='hard_sigmoid'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.LSTM(200, activation='sigmoid', recurrent_activation='hard_sigmoid'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

print ('Compiling...')
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(x_train, y_train, batch_size = 64, epochs=50, validation_split = 0.1, verbose = 1)

val_loss, val_acc = model.evaluate(x_test, y_test)
print(val_loss)
print(val_acc)

end = time.time()
print("Total Time -->", end - start)

Compiling...


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [ ]:
score, acc = model.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
def evaluateWordToVec():
    Word2Vec_model = Word2Vec.load(modelfileName)
    print("Loaded successfully")
    #print(model.wv['FEB0FF999E500000'])
    print(len(Word2Vec_model.wv['FEB0FF999E500000']))

    #print(model.wv['000200000000002A'])
    print(len(Word2Vec_model.wv['000200000000002A']))
    
    #print(model.wv['FEB0FF999E500000'] + model.wv['000200000000002A'])
    
    v = Word2Vec_model.wv['FEB0FF999E500000'] + Word2Vec_model.wv['000200000000002A']
    print(type(v))
    #print(v/2)
evaluateWordToVec()